In [1]:
import h5py

def explore_h5(file_path):
    """
    Mở file .h5 và in ra cấu trúc:
      - Các group và dataset
      - Hình dạng (shape) và kiểu dữ liệu (dtype) của mỗi dataset
    """
    with h5py.File(file_path, 'r') as f:
        print(f"Root groups/datasets in '{file_path}':")
        for key in f.keys():
            print(f"  - {key}: {type(f[key]).__name__}")
        
        def print_item(name, obj):
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset: {name}")
                print(f"  shape: {obj.shape}")
                print(f"  dtype: {obj.dtype}")
            elif isinstance(obj, h5py.Group):
                print(f"Group:   {name}")
        
        print("\nDetailed structure:")
        f.visititems(print_item)

if __name__ == "__main__":
    # Thay 'your_file.h5' bằng đường dẫn tới file của bạn
    explore_h5('datasets/ACDC/ACDC_training_slices/patient001_frame01_slice_0.h5')


Root groups/datasets in 'datasets/ACDC/ACDC_training_slices/patient001_frame01_slice_0.h5':
  - image: Dataset
  - label: Dataset
  - scribble: Dataset

Detailed structure:
Dataset: image
  shape: (256, 216)
  dtype: float32
Dataset: label
  shape: (256, 216)
  dtype: uint8
Dataset: scribble
  shape: (256, 216)
  dtype: uint16
